In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
import re
import string
import os,glob
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import keras
from nltk.corpus import stopwords

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
excel_file = r'/content/drive/MyDrive/Embeddings/Fully compiled dataset Sentence New.xlsx'
df = pd.read_excel(excel_file)
mapping = {'Plagiarized': 1, 'Unplagiarized': 0}
df['Type'] = df['Type'].replace(mapping)
df_type_0 = df[df['Type'] == 0]
df_type_1 = df[df['Type'] == 1]

# Shuffle both DataFrames
df_type_0 = df_type_0.sample(frac=1).reset_index(drop=True)
df_type_1 = df_type_1.sample(frac=1).reset_index(drop=True)

# Calculate the length of the longest DataFrame
max_length = max(len(df_type_0), len(df_type_1))

# Create a new DataFrame to hold the alternating rows
randomized_df = pd.DataFrame(columns=df.columns)

# Iterate through both DataFrames and append rows alternately until one of them runs out
for i in range(max_length):
    if i < len(df_type_0):
        randomized_df = randomized_df.append(df_type_0.iloc[i], ignore_index=True)
    if i < len(df_type_1):
        randomized_df = randomized_df.append(df_type_1.iloc[i], ignore_index=True)
mapping = {'Plagiarized': 1, 'Unplagiarized': 0}
randomized_df['Type'] = randomized_df['Type'].replace(mapping)
randomized_df.drop('ID', axis=1, inplace=True)
randomized_df

<ipython-input-3-5e724572395f>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  randomized_df = randomized_df.append(df_type_0.iloc[i], ignore_index=True)
<ipython-input-3-5e724572395f>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  randomized_df = randomized_df.append(df_type_1.iloc[i], ignore_index=True)


,Text,Type
0,* تیسرا دور۔۔۔۔ اصلاحی تحریک۔علی گڑھ کا قیام ا...,0
1,بہت سی قیمتی کتب کا ذخیرہ ، احادیث کی تخریج،او...,1
2,آپ ﷺ کے فرامین اور عملی زندگی سے ہمیں عمل پیہ...,0
3,۔ مجموعی طور پر، ہم اپنی زندگی میں تعلیم کی اہ...,1
4,قرآن کریم کی تعلیم پر غور کرنے سے معلوم ہوتا ...,0
...,...,...
2515,علامہ اقبال 9 نومبر 1877 کو سیالکوٹ میں پیدا ہ...,1
2516,آج کے انسان نے جتنی بھی ترقی کی ہے۔اپنے علم و ...,0
2517,* خود اُٹھ کر پانی کا گلاس پی لینا ہمیں بوجھ ...,1
2518,اور محنت کرنے کی ترغیب کے حوالے سے ارشادات ربا...,0


In [ ]:
# prompt: count the data in randomized_df,grouped by lable

grouped_data = randomized_df.groupby('Type').count()
print(grouped_data)


      Text
Type      
0     1260
1     1260


In [ ]:
randomized_df['Text'] = randomized_df['Text'].apply(lambda x: " ".join(x.lower() for x in x.split())) #convert text into lower case

randomized_df['Text'] = randomized_df['Text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])    # Remove URLs

#dataset_df['Text'] = dataset_df['Text'].apply(lambda x: " ".join(x for x in x.split() if x[0]!="@"))

#dataset_df['Text'] = dataset_df['Text'].apply(lambda x: " ".join(x for x in x.split() if x[0]!="#"))

#dataset_df['Text'] = dataset_df['Text'].str.replace('\d+', '')  #Remove digits

randomized_df['Text'] = randomized_df['Text'].map(lambda x: re.sub(r'\W+', ' ', x)) # Remove Punctuation marks

#stop = stopwords.words('english')
#dataset_df['Text'] = dataset_df['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop)) #Remove stop words

print("\nDataset After Text Preprocessing:")
print("=================================================\n")
randomized_df.head(7)


Dataset After Text Preprocessing:



,Text,Type
0,تیسرا دور اصلاحی تحریک علی گڑھ کا قیام اور او...,0
1,بہت سی قیمتی کتب کا ذخیرہ احادیث کی تخریج اور ...,1
2,ا پ ﷺ کے فرامین اور عملی زندگی سے ہمیں عمل پیہ...,0
3,مجموعی طور پر ہم اپنی زندگی میں تعلیم کی اہمی...,1
4,قرا ن کریم کی تعلیم پر غور کرنے سے معلوم ہوتا ...,0
5,یہ ٹیکنالوجی دنیا کے اربوں انسانوں کو جدید اور...,1
6,قائد اعظم نے خود فرمایا کہ پاکستان اللہ کی مدد...,0


In [ ]:
!pip install --upgrade gensim -q
!pip install matplotlib -q

In [ ]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string

In [ ]:
gensim.__version__

'4.3.2'

In [ ]:
np.random.seed(42)

In [ ]:
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
#wv = api.load('fasttext-wiki-news-subwords-300')

In [ ]:
#wv.save('/content/drive/MyDrive/Embeddings/fasttext-wiki-news-subwords-300.kv')

In [ ]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load('/content/drive/MyDrive/Embeddings/fasttext-wiki-news-subwords-300.kv')

In [ ]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [ ]:
sent_vec("I am happy")

array([ 3.53637545e-02, -1.11509564e-02,  7.97317783e-02,  3.29024217e-02,
       -1.31729889e-01,  4.23039452e-02,  2.65068894e-02,  2.49073337e-01,
       -1.91124429e-02, -4.37758670e-02, -4.74023335e-02,  9.58162215e-02,
        8.89166755e-03,  3.85325555e-02,  4.87841107e-02, -3.05376665e-02,
       -1.36644464e-02, -1.58806656e-02, -9.82083556e-02, -1.54814451e-02,
       -6.80230670e-02,  4.00955551e-02,  1.10968043e-02,  8.55041590e-02,
        5.40010002e-02,  4.32255214e-02,  5.81695451e-02,  7.80289984e-02,
       -7.25063998e-02,  2.00253549e-02, -2.14306667e-02,  1.45389551e-02,
        3.86895554e-02,  2.87967767e-02,  4.18810345e-02, -1.84841330e-01,
       -1.02395222e-01,  7.90986703e-03,  5.62034440e-02,  5.68341678e-02,
       -1.56207219e-02,  1.47988666e-01, -3.62244445e-02, -9.35089994e-02,
        4.66333288e-04, -2.99166598e-03,  6.95537796e-03,  6.51063329e-02,
       -3.80104439e-02,  1.00723646e-01,  1.19178443e-01, -2.45741221e-02,
        7.00977794e-02, -

In [ ]:
randomized_df['vec'] = randomized_df['Text'].apply(sent_vec)

In [ ]:
randomized_df.head()

,Text,Type,vec
0,تیسرا دور اصلاحی تحریک علی گڑھ کا قیام اور او...,0,"[-0.05696586916223168, 0.008066956802056387, -..."
1,بہت سی قیمتی کتب کا ذخیرہ احادیث کی تخریج اور ...,1,"[-0.06061794179491699, 0.010298921867652099, -..."
2,ا پ ﷺ کے فرامین اور عملی زندگی سے ہمیں عمل پیہ...,0,"[-0.05307925016395459, 0.005630965486291594, -..."
3,مجموعی طور پر ہم اپنی زندگی میں تعلیم کی اہمی...,1,"[-0.06305266266266249, 0.009854421179175147, -..."
4,قرا ن کریم کی تعلیم پر غور کرنے سے معلوم ہوتا ...,0,"[-0.059257254476937714, 0.009591206893567879, ..."


In [ ]:
X = randomized_df['vec'].to_list()
y = randomized_df['Type'].to_list()

In [ ]:
  X[0]

array([-5.69658692e-02,  8.06695680e-03, -6.12139428e-03, -3.57660183e-02,
        3.15841608e-03,  5.18062565e-02,  8.31267549e-03,  1.35100912e-01,
        2.25291714e-02,  5.56362824e-02,  3.23675373e-02,  6.96028431e-02,
       -3.70006469e-02,  2.65260162e-02,  2.34846100e-02, -6.78388551e-02,
       -1.04259342e-01,  2.52574913e-02, -5.47208246e-02, -2.87250746e-02,
       -1.04887990e-02,  3.99895623e-02,  6.68943268e-02, -4.24022333e-02,
        3.94513245e-02,  2.52452291e-02,  9.13138133e-02,  2.72283704e-02,
       -1.15961658e-01, -6.52598559e-02, -1.89979159e-01, -6.50398356e-02,
        5.54172330e-02,  5.47511387e-02,  2.79036675e-02, -1.27461324e-02,
       -7.18379743e-02,  1.04708297e-01,  8.98560977e-03,  1.73276991e-02,
        4.68675328e-03,  9.10965846e-02,  2.66788589e-03,  1.82238599e-02,
       -4.51459879e-02, -3.28998059e-02, -8.87009258e-02, -3.62206441e-02,
        7.42390342e-02,  4.04236344e-02,  3.54280263e-02, -4.72276644e-02,
       -1.85701664e-02, -

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier_LR = LogisticRegression()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

# Assuming X and y are your feature matrix and target vector respectively


# Initialize StratifiedKFold with 5 folds
skf = StratifiedKFold(n_splits=5)



In [ ]:
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='weighted')
}

In [ ]:
# Perform cross-validation
cv_results = cross_validate(classifier_LR, X, y, cv=skf, scoring=scoring)


In [ ]:
# Display the cross-validation scores
print("Mean Accuracy:", cv_results['test_accuracy'].mean())
print("Mean Precision:", cv_results['test_precision'].mean())
print("Mean Recall:", cv_results['test_recall'].mean())
print("Mean F1 Score:", cv_results['test_f1_score'].mean())

Mean Accuracy: 0.5488095238095237
Mean Precision: 0.5492615954645218
Mean Recall: 0.5488095238095237
Mean F1 Score: 0.5477618705397793


In [ ]:
dtree = DecisionTreeClassifier()

In [ ]:
# Perform cross-validation
cv_results = cross_validate(dtree, X, y, cv=skf, scoring=scoring)


In [ ]:
# Display the cross-validation scores
print("Mean Accuracy:", cv_results['test_accuracy'].mean())
print("Mean Precision:", cv_results['test_precision'].mean())
print("Mean Recall:", cv_results['test_recall'].mean())
print("Mean F1 Score:", cv_results['test_f1_score'].mean())

Mean Accuracy: 0.5698412698412698
Mean Precision: 0.5700801843328802
Mean Recall: 0.5698412698412698
Mean F1 Score: 0.5695264400892411


In [ ]:
# Initialize classifiers
logreg = LogisticRegression()
dtree = DecisionTreeClassifier()
rf = RandomForestClassifier()
svm = SVC()
knn = KNeighborsClassifier()
nb = GaussianNB()
lda = LinearDiscriminantAnalysis()
mlp = MLPClassifier(max_iter=1000)  # Adjust max_iter based on convergence
xgb = GradientBoostingClassifier()

In [ ]:
# Create a voting classifier
voting_clf = VotingClassifier(estimators=[
    ('logreg', logreg),
    ('dtree', dtree),
    ('rf', rf),
    ('svm', svm),
    ('knn', knn),
    ('nb', nb),
    ('lda', lda),
    ('mlp', mlp),
    ('xgb', xgb)
], voting='hard')

In [ ]:
# Initialize classifiers list
classifiers = [logreg, dtree, rf, svm, knn, nb, lda, mlp, xgb, voting_clf]

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
n_folds = 5

# Initialize KFold object
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

In [ ]:
# Iterate over classifiers
for clf in classifiers:
    # Perform k-fold cross-validation
    scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')

    # Calculate mean accuracy
    mean_accuracy = np.mean(scores)

    # Print mean accuracy
    print(f"Mean accuracy for {clf.__class__.__name__}: {mean_accuracy:.4f}")

Mean accuracy for LogisticRegression: 0.5488
Mean accuracy for DecisionTreeClassifier: 0.5667
Mean accuracy for RandomForestClassifier: 0.6377
Mean accuracy for SVC: 0.5357
Mean accuracy for KNeighborsClassifier: 0.5817
Mean accuracy for GaussianNB: 0.5563
Mean accuracy for LinearDiscriminantAnalysis: 0.6655
Mean accuracy for MLPClassifier: 0.5575
Mean accuracy for GradientBoostingClassifier: 0.6468
Mean accuracy for VotingClassifier: 0.6091


In [ ]:
# Iterate over classifiers
for clf in classifiers:
    # Perform k-fold cross-validation
    #scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')

    # Calculate mean accuracy
    #mean_accuracy = np.mean(scores)

    # Print mean accuracy
    #print(f"Mean accuracy for {clf.__class__.__name__}: {mean_accuracy:.4f}")

    scores_accuracy = cross_val_score(clf, X, y, cv=skf, scoring='accuracy')
    scores_recall = cross_val_score(clf, X, y, cv=skf, scoring='recall')
    scores_precision = cross_val_score(clf, X, y, cv=skf, scoring='precision')
    scores_f1 = cross_val_score(clf, X, y, cv=skf, scoring='f1')

    # Calculate mean metrics
    mean_accuracy = np.mean(scores_accuracy)
    mean_recall = np.mean(scores_recall)
    mean_precision = np.mean(scores_precision)
    mean_f1 = np.mean(scores_f1)

    # Print metrics
    print(f"Metrics for {clf.__class__.__name__}:")
    print(f"**Accuracy:** {mean_accuracy:.4f}")
    print(f"**Recall:** {mean_recall:.4f}")
    print(f"**Precision:** {mean_precision:.4f}")
    print(f"**F1 Score:** {mean_f1:.4f}\n")


Metrics for LogisticRegression:
**Accuracy:** 0.5488
**Recall:** 0.5230
**Precision:** 0.5518
**F1 Score:** 0.5361

Metrics for DecisionTreeClassifier:
**Accuracy:** 0.5663
**Recall:** 0.5635
**Precision:** 0.5859
**F1 Score:** 0.5669

Metrics for RandomForestClassifier:
**Accuracy:** 0.6544
**Recall:** 0.6270
**Precision:** 0.6582
**F1 Score:** 0.6433

Metrics for SVC:
**Accuracy:** 0.5448
**Recall:** 0.5087
**Precision:** 0.5492
**F1 Score:** 0.5276

Metrics for KNeighborsClassifier:
**Accuracy:** 0.5833
**Recall:** 0.5683
**Precision:** 0.5859
**F1 Score:** 0.5766

Metrics for GaussianNB:
**Accuracy:** 0.5567
**Recall:** 0.4460
**Precision:** 0.5731
**F1 Score:** 0.5014

Metrics for LinearDiscriminantAnalysis:
**Accuracy:** 0.6639
**Recall:** 0.6524
**Precision:** 0.6680
**F1 Score:** 0.6595

Metrics for MLPClassifier:
**Accuracy:** 0.5619
**Recall:** 0.6492
**Precision:** 0.5700
**F1 Score:** 0.5363

Metrics for GradientBoostingClassifier:
**Accuracy:** 0.6536
**Recall:** 0.6294
**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Define hyperparameter grids for each classifier
param_grid = {
    'LogisticRegression': {
        'C': [0.1, 1, 10, 100],
        'penalty': ['l1', 'l2']
    },
    'DecisionTreeClassifier': {
        'max_depth': [None, 5, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'RandomForestClassifier': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 5, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'SVC': {
        'C': [0.1, 1, 10, 100],
        'gamma': [0.1, 1, 10]
    },
    'KNeighborsClassifier': {
        'n_neighbors': [3, 5, 10],
        'weights': ['uniform', 'distance']
    },
    'GaussianNB': {},

    'GradientBoostingClassifier': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.5],
        'max_depth': [3, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'VotingClassifier': {}  # No hyperparameters to tune for VotingClassifier
}

In [ ]:
# Perform random search for each classifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

for clf in classifiers:
    clf_name = clf.__class__.__name__
    print(f"Optimizing hyperparameters for {clf_name}...")

    if clf_name in param_grid:
        param_dist = param_grid[clf_name]
        random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42)
        random_search.fit(X, y)

        print("Best parameters found:")
        print(random_search.best_params_)

        # Perform cross-validation with best parameters
        best_clf = random_search.best_estimator_
        scores_accuracy = cross_val_score(best_clf, X, y, cv=5, scoring='accuracy')
        scores_recall = cross_val_score(best_clf, X, y, cv=5, scoring='recall')
        scores_precision = cross_val_score(best_clf, X, y, cv=5, scoring='precision')
        scores_f1 = cross_val_score(best_clf, X, y, cv=5, scoring='f1')

        # Calculate mean metrics
        mean_accuracy = np.mean(scores_accuracy)
        mean_recall = np.mean(scores_recall)
        mean_precision = np.mean(scores_precision)
        mean_f1 = np.mean(scores_f1)

        # Print mean metrics
        print(f"Mean Accuracy: {mean_accuracy:.4f}")
        print(f"Mean Recall: {mean_recall:.4f}")
        print(f"Mean Precision: {mean_precision:.4f}")
        print(f"Mean F1 Score: {mean_f1:.4f}\n")
    else:
        print("No hyperparameters to tune.")


Optimizing hyperparameters for LogisticRegression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown i

Best parameters found:
{'penalty': 'l2', 'C': 100}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Mean Accuracy: 0.6167
Mean Recall: 0.5960
Mean Precision: 0.6222
Mean F1 Score: 0.6084

Optimizing hyperparameters for DecisionTreeClassifier...
Best parameters found:
{'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 5}
Mean Accuracy: 0.5917
Mean Recall: 0.4802
Mean Precision: 0.6277
Mean F1 Score: 0.5332

Optimizing hyperparameters for RandomForestClassifier...
Best parameters found:
{'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': None}
Mean Accuracy: 0.6567
Mean Recall: 0.6524
Mean Precision: 0.6615
Mean F1 Score: 0.6466

Optimizing hyperparameters for SVC...
Best parameters found:
{'gamma': 10, 'C': 100}
Mean Accuracy: 0.6734
Mean Recall: 0.6540
Mean Precision: 0.6805
Mean F1 Score: 0.6668

Optimizing hyperparameters for KNeighborsClassifier...


...

Best parameters found:
{'weights': 'distance', 'n_neighbors': 10}
Mean Accuracy: 0.6016
Mean Recall: 0.5611
Mean Precision: 0.6098
Mean F1 Score: 0.5841

Optimizing hyperparameters for GaussianNB...
Best parameters found:
{}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Mean Accuracy: 0.5567
Mean Recall: 0.4460
Mean Precision: 0.5731
Mean F1 Score: 0.5014

Optimizing hyperparameters for LinearDiscriminantAnalysis...
No hyperparameters to tune.
Optimizing hyperparameters for MLPClassifier...
No hyperparameters to tune.
Optimizing hyperparameters for GradientBoostingClassifier...
Best parameters found:
{'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 10, 'learning_rate': 0.5}
Mean Accuracy: 0.6766
Mean Recall: 0.6421
Mean Precision: 0.6671
Mean F1 Score: 0.6548

Optimizing hyperparameters for VotingClassifier...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters found:
{}
Mean Accuracy: 0.6052
Mean Recall: 0.5738
Mean Precision: 0.6222
Mean F1 Score: 0.5926

